Docs:
- https://huggingface.co/docs/optimum-onnx/onnxruntime/package_reference/modeling_ort#optimum.onnxruntime.ORTModelForFeatureExtraction
- https://huggingface.co/optimum/all-MiniLM-L6-v2/tree/main
- https://huggingface.co/docs/transformers/en/serialization
- https://github.com/onnx/tutorials

Notes:
- it seems that optimum-onnx is deprecated

#### Test with transformers pipeline

In [ ]:
import numpy as np

In [ ]:
import certifi

certifi.where()

In [ ]:
!uv pip list | grep transformers

In [ ]:
!uv pip list | grep numpy

In [ ]:
!uv pip list | grep optimum

In [ ]:
model_id = "optimum/all-MiniLM-L6-v2"

In [ ]:
from transformers import AutoTokenizer, pipeline



tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# from transformers import FeatureExtractionMixin

# extractor = FeatureExtractionMixin.from_pretrained(model_id)

#### Test with onnx runtime

In [ ]:
# from transformers import AutoTokenizer, pipeline
# from optimum.onnxruntime import ORTModelForFeatureExtraction


# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = ORTModelForFeatureExtraction.from_pretrained(model_id)
# onnx_extractor = pipeline("feature-extraction", model=model, tokenizer=tokenizer)

# text = "My name is Philipp and I live in Germany."
# pred = onnx_extractor(text)

In [ ]:
# from optimum.onnxruntime import ORTModelForFeatureExtraction
# from optimum.onnxruntime import ORTModelForSequenceClassification

# model = ORTModelForFeatureExtraction.from_pretrained(model_id)

### 

In [ ]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("distilbert_base_uncased_squad_onnx")
model = ORTModelForQuestionAnswering.from_pretrained("distilbert_base_uncased_squad_onnx")
inputs = tokenizer("What am I using?", "Using DistilBERT with ONNX Runtime!", return_tensors="pt")
outputs = model(**inputs)

#### Test with ONNX Session

In [ ]:
import onnxruntime

# MODEL_PATH = "../../../apps/web/model.onnx"
MODEL_PATH = "../../../apps/web/model/model.onnx"

options = onnxruntime.SessionOptions()
options.enable_profiling=True
session = onnxruntime.InferenceSession(MODEL_PATH, sess_options=options) # optimum/all-MiniLM-L6-v2.onnx

In [ ]:
print([inp.name for inp in session.get_inputs()])
print([inp.shape for inp in session.get_inputs()])
print([out.name for out in session.get_outputs()])

In [ ]:
from transformers import AutoTokenizer

TOKENIZER_DIR = "../../../apps/web/model/"

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_DIR)

In [ ]:
text = "testing some sentence"
encoded = tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors="np")

onnx_inputs = {
    "input_ids": encoded["input_ids"].astype(np.int64),
    "attention_mask": encoded["attention_mask"].astype(np.int64),
    "token_type_ids": encoded["token_type_ids"].astype(np.int64),
}

token_embeddings, sentence_embeddings = session.run(None, onnx_inputs)

In [ ]:
import onnx

model = onnx.load(MODEL_PATH)
print("Inputs:", [input.name for input in model.graph.input])
print("Outputs:", [output.name for output in model.graph.output])

In [ ]:
# TODO: quantize